# 2D numerical experiment of the inverted pendulum

In [1]:
import gpytorch
import torch
from src.TVBO import TimeVaryingBOModel
from src.objective_functions_LQR import lqr_objective_function_2D

### Hyperparameters

In [2]:
# parameters regarding the objective function
objective_function_options = {'objective_function': lqr_objective_function_2D,

                              # optimize the last two entries of the feedback gain
                              'spatio_dimensions': 2,

                              # approximate noise level from the objective function
                              'noise_lvl': 0.005,

                              # feasible set for the optimization
                              'feasible_set': torch.tensor([[-62.5, -5], [-12.5, -1]],
                                                           dtype=torch.float),

                              # initial feasible set consisting of only controllers
                              'initial_feasible_set': torch.tensor([[-50, -4], [-25, -2]],
                                                                   dtype=torch.float),

                              # scaling \theta to have approximately equal lengthscales in each dimension
                              'scaling_factors': torch.tensor([3, 1 / 4])}

# parameters regarding the model
model_options = {'constrained_dimensions': None,  # later specified for each variation
                 'forgetting_type': None,  # later specified for each variation
                 'forgetting_factor': None,  # later specified for each variation

                 # specification for the constraints  (cf. Agrell 2019)
                 'nr_samples': 10000,
                 'xv_points_per_dim': 4,  # VOPs per dimensions
                 'truncation_bounds': [0, 2],

                 # specification of prior
                 'prior_mean': 0.,
                 'lengthscale_constraint': gpytorch.constraints.Interval(0.5, 6),
                 'lengthscale_hyperprior': gpytorch.priors.GammaPrior(6, 1 / 0.3),
                 'outputscale_constraint_spatio': gpytorch.constraints.Interval(0, 20),
                 'outputscale_hyperprior_spatio': None, }

### Specify variations

In [3]:
# UI -> UI-TVBO, B2P_OU -> TV-GP-UCB
variations = [
    # in the paper color green
    {'forgetting_type': 'UI', 'forgetting_factor': 0.03, 'constrained_dims': [0, 1]},

    # in the paper color blue
    {'forgetting_type': 'UI', 'forgetting_factor': 0.03, 'constrained_dims': []},

    # in the paper color purple
    {'forgetting_type': 'B2P_OU', 'forgetting_factor': 0.03, 'constrained_dims': [0, 1]},

    # in the paper color red
    {'forgetting_type': 'B2P_OU', 'forgetting_factor': 0.03, 'constrained_dims': []}, ]

### Start optimization

In [4]:
trials_per_variation = 25  # number of different runs
for variation in variations:

    # update variation specific parameters
    model_options['forgetting_type'] = variation['forgetting_type']
    model_options['forgetting_factor'] = variation['forgetting_factor']
    model_options['constrained_dimensions'] = variation['constrained_dims']

    tvbo_model = TimeVaryingBOModel(objective_function_options=objective_function_options,
                                    model_options=model_options,
                                    post_processing_options={},
                                    add_noise=False, )  # noise is added during the simulation of the pendulum

    # specify name to safe results
    method_name = model_options['forgetting_type']
    forgetting_factor = model_options['forgetting_factor']
    string = 'constrained' if model_options['constrained_dimensions'] else 'unconstrained'
    NAME = f"{method_name}_2DLQR_{string}_forgetting_factor_{forgetting_factor}".replace('.', '_')

    # run optimization
    for trial in range(1, trials_per_variation + 1):
        tvbo_model.run_TVBO(n_initial_points=30,
                            time_horizon=300,
                            safe_name=NAME,
                            trial=trial, )
    print('Finished.')

Initialization


C:\Users\11659\AppData\Local\Continuum\anaconda3\envs\tvbo\lib\site-packages\botorch\fit.py:148: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:2228.)
  warnings.warn(w.message, w.category)


Time for optimizing aquisition functions: 0.225s.

Timestep   30 of Trail  1.
Parameter name: noise value = 0.000
Parameter name: outputscale value = 1.000
Parameter name: lengthscale 0 value = 3.564
Parameter name: lengthscale 1 value = 3.921
Start minimax algorithm (Python implementation) for 10000 samples (D=32)...
Time taken for 10000 samples: 0.53s.
Time for optimizing aquisition functions: 0.893s.
Time for optimizing aquisition functions: 0.365s.
Time taken for timestep   30: 1.347s.

Timestep   31 of Trail  1.
Parameter name: noise value = 0.000
Parameter name: outputscale value = 1.000
Parameter name: lengthscale 0 value = 3.233
Parameter name: lengthscale 1 value = 3.796
Start minimax algorithm (Python implementation) for 10000 samples (D=32)...
Time taken for 10000 samples: 2.13s.
Time for optimizing aquisition functions: 2.486s.
Time for optimizing aquisition functions: 0.269s.
Time taken for timestep   31: 2.804s.

Timestep   32 of Trail  1.
Parameter name: noise value = 0.

RuntimeError: Attempting to manually set a parameter value that is out of bounds of its current constraints, Interval(5.000E-01, 6.000E+00). Most likely, you want to do the following:
 likelihood = GaussianLikelihood(noise_constraint=gpytorch.constraints.GreaterThan(better_lower_bound))